In [ ]:
# ! pip install torchtext==0.6.0 pyvi spacy https://gitlab.com/trungtv/vi_spacy/-/raw/master/vi_core_news_lg/dist/vi_core_news_lg-0.0.1.tar.gz rouge_score
# ! python -m spacy download en_core_web_sm

In [ ]:
# ! git clone https://github.com/namnh194/transformer_pointer_generator_network.git

In [ ]:
# refer: github.com/pbcquoc
from dataset import read_data, create_fields, create_dataset
from model import Transformer
from train_utils import ScheduledOptim, LabelSmoothingLoss, step, validiate
from inference import rouge_score, translate_sentence
import torch, tqdm
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import datasets

dataset = datasets.load_dataset("nam194/vietnews")
dataset

In [ ]:
opt = {
    'src_lang': 'vi_core_news_lg',
    'trg_lang': 'vi_core_news_lg',#'vi_spacy_model',
    'en_max_strlen': 256,
    'de_max_strlen': 64,
    'batchsize': 1000,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'd_model': 512,
    'n_layers': 6,
    'heads': 8,
    'dropout': 0.1,
    'lr': 0.0001,
    'epochs': 20,
    'printevery': 200,
    'k': 5,
}

In [ ]:
train_src_data, train_trg_data = read_data(dataset, 'train', 'article', 'abstract')
valid_src_data, valid_trg_data = read_data(dataset, 'validation', 'article', 'abstract')

SRC, TRG = create_fields(opt['src_lang'], opt['trg_lang'], opt['en_max_strlen'], opt['de_max_strlen'])
train_iter = create_dataset(train_src_data, train_trg_data, opt['batchsize'], opt['device'], SRC, TRG, istrain=True)
valid_iter = create_dataset(valid_src_data, valid_trg_data, opt['batchsize'], opt['device'], SRC, TRG, istrain=False)

In [ ]:
src_pad = SRC.vocab.stoi['<pad>']
trg_pad = TRG.vocab.stoi['<pad>']
src_pad, trg_pad

In [ ]:
SRC.preprocess('tôi không còn là sinh viên     đại   học  BKHN')
# SRC.vocab.stoi['sinh_viên']

In [ ]:
from train_utils import create_masks

src = batch.src.transpose(0, 1)
trg = batch.trg.transpose(0, 1)
trg_input = trg[:, :-1]
src_mask, trg_mask = create_masks(src, trg_input, src_pad, trg_pad, device='cpu')
# preds = model(src, trg_input, src_mask, trg_mask)
print(src.shape, trg.shape, trg_input.shape)
print(src_mask.shape, trg_mask.shape, src_mask, trg_mask)

In [ ]:
en_config = {
    "vocab_size": len(SRC.vocab),
    "max_seq_len": opt['en_max_strlen'],
    "d_model": opt['d_model'],
    "n_heads": opt['heads'],
    "d_ff": 2048,
    "num_layer": opt['n_layers'],
    "dropout": opt['dropout']}
de_config = {
    "vocab_size": len(TRG.vocab),
    "max_seq_len": opt['de_max_strlen'],
    "d_model": opt['d_model'],
    "n_heads": opt['heads'],
    "d_ff": 2048,
    "num_layer": opt['n_layers'],
    "dropout": opt['dropout']}

batch_size = opt['batchsize']
en_seq_len = en_config["max_seq_len"]
de_seq_len = de_config["max_seq_len"]
en_vocab_size = en_config["vocab_size"]
de_vocab_size = de_config["vocab_size"]

model = Transformer(en_config, de_config)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
model = model.to(opt['device'])

In [ ]:
import gc
from accelerate import Accelerator

optimizer = ScheduledOptim(
        torch.optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-09),
        0.2, opt['d_model'], 4000)
criterion = LabelSmoothingLoss(len(TRG.vocab), padding_idx=trg_pad, smoothing=0.1)

model, optimizer, train_iter, valid_iter = Accelerator().prepare(
    model, optimizer, train_iter, valid_iter)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
!wandb login e67e803979133f932a24dac4dedb24348671c1ba

In [ ]:
import wandb
wandb.init(
    # set the wandb project where this run will be logged
    project="transformer_VNDS",
    
    # track hyperparameters and run metadata
    config=opt
)

In [ ]:
import time

for epoch in range(opt['epochs']):
    total_loss = 0
    
    for i, batch in enumerate(train_iter): 
        s = time.time()
        loss = step(model, optimizer, batch, criterion, src_pad, trg_pad)
        
        total_loss += loss
        
        if (i + 1) % opt['printevery'] == 0:
            avg_loss = total_loss/opt['printevery']
            wandb.log({"train_loss": avg_loss, "step": (i+1)+epoch*3097})
            print('epoch: {:03d} - iter: {:05d} - train loss: {:.4f} - time: {:.4f}'.format(epoch, i, avg_loss, time.time()- s))
            total_loss = 0
            
    s = time.time()
    valid_loss = validiate(model, valid_iter, criterion, src_pad, trg_pad)
    score = rouge_score(valid_src_data[:500], valid_trg_data[:500], model, SRC, TRG, opt['device'], opt['k'], opt['max_strlen'])
    print('epoch: {:03d} - iter: {:05d} - valid loss: {:.4f} - bleu score: {:.4f} - time: {:.4f}'.format(epoch, i, valid_loss, score, time.time() - s))

In [139]:
batch = next(iter(valid_iter)).src[:,0]
print(batch.shape)
sentence = []

for i in batch:
    if i not in [0,1]:
        sentence += [SRC.vocab.itos[i]]
sentence = ' '.join(sentence).replace('_',' ')
print(sentence)

trans_sent = translate_sentence(sentence, model, SRC, TRG, opt['device'], opt['k'], opt['de_max_strlen'])
trans_sent

torch.Size([256])
Khuôn viên Đại học Stanford , bang California , Mỹ . Các nguồn tin giấu tên hôm qua tiết lộ một gia đình Trung Quốc đã trả 6,5 triệu USD cho William Rick Singer , chủ sở hữu công ty tư vấn đào tạo dự bị đại học Edge College & Career Network , để đảm bảo cho con gái của họ có một suất tại Đại học Stanford , Mỹ . Đây được cho là khoản tiền lớn nhất mà các phụ huynh trả cho Singer , kẻ cầm đầu đường dây chạy vào các trường đại học danh giá của Mỹ . Sau Chiến dịch Versity Blues kéo dài một năm , các công tố viên liên bang Mỹ hồi tháng 3 công bố cách thức hoạt động của đường dây này , đồng thời nộp hồ sơ truy tố lên Bộ Tư pháp . Truyền thông Mỹ xác định nữ sinh Trung Quốc được bố mẹ chi tiền chạy vào Đại học Stanford có tên là Yusi " Molly " Zhao . Gia đình Zhao sống tại Bắc Kinh và được một cố vấn tài chính của ngân hàng Morgan Stanley giới thiệu với Singer . Nữ sinh này được nhận vào Stanford năm 2017 nhờ được Singer giới thiệu là một vận động viên đua thuyền đầy hứa hẹn

'chứng_thư chứng_thư chứng_thư sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư Phúc_Cường sinh_thời Phúc_Cường sinh_thời chứng_thư chứng_thư chứng_thư sinh_thời Phúc_Cường Phúc_Cường Phúc_Cường Vương_Chí_Dân chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư'

0 torch.Size([256, 11])


In [148]:
for i, batch in enumerate(train_iter):
    if batch.src.shape[-1] != 1:
        print(i, batch.src.shape)
        break

for idx in range(batch.src.shape[-1]):
    print('idx: ',idx)
    sentence = []
    for i in batch.src[:,idx]:
        if i not in [0,1]:
            sentence += [SRC.vocab.itos[i].replace('_',' ')]
    print(' '.join(sentence))
    print('---------------------------------------------------')

0 torch.Size([256, 2])
idx:  0
Ngày 29 - 11 , trao đổi với Tuổi Trẻ về việc Sở Tài nguyên - môi trường Hà Nội lấy ý kiến góp ý của các sở ngành , quận huyện về phương án bảo vệ khoáng sản chưa khai thác , ông Nguyễn Minh Mười , phó giám đốc Sở Tài nguyên - môi trường Hà Nội khẳng định chủ trương chung là không khuyến khích khai thác khoáng sản . Theo phương án bảo vệ khoáng sản chưa khai thác , Sở Tài nguyên - môi trường Hà Nội cho biết trên địa bàn thành phố có một số loại khoáng sản chính và có triển vọng khai thác là đá xây dựng , cát xây dựng và cát san lấp , sét gạch ngói , than bùn và puzolan . Đá xây dựng được phân bố dọc theo ranh giới phía tây , tiếp giáp với Hoà Bình từ Núi Chẹ , Đá Chông ( huyện Ba Vì ) qua Phú Mãn , Núi Voi ( huyện Quốc Oai ) đến Mỹ Đức và một vài mỏ nhỏ ở ranh giới xã Minh Phú , xã Nam Sơn ( huyện Sóc Sơn ) . Cát san lấp , cát xây dựng phân bố tại lưu vực sông Hồng , sông Đà , sông Đáy , sông Cà lồ đi qua các quận , huyện như Sóc Sơn , Mê Linh , Đông Anh ,